In [1]:
import csv
import numpy as np
import glob
import json
import matplotlib.pyplot as plt
from PIL import Image
import open3d as o3d # 0.14 has ray casting
import point_cloud_utils as pcu

def convert_to_o3dpcd(points, color=None):
    if type(points) == list:
        pcds = []
        for pointcloud in points:
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(pointcloud[:,:3])
            pcds.append(pcd)
        return pcds
    else:
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points[:,:3])
        if color:
            pcd.paint_uniform_color(color)
        return pcd
    
def create_o3daxis(origin, size=1):
    return o3d.geometry.TriangleMesh.create_coordinate_frame(size=size, origin=origin)

def boxpts_to_o3dbox(boxpts):
    o3dpts = o3d.utility.Vector3dVector(boxpts)
    o3dbox = o3d.geometry.OrientedBoundingBox().create_from_points(o3dpts)
    o3dbox.color = np.array([1,0,0])
    return o3dbox

def get_minmax(pts):
    return {'max_x':max(pts[:,0]), 'min_x':min(pts[:,0]),
            'max_y':max(pts[:,1]), 'min_y':min(pts[:,1]),
            'max_z':max(pts[:,2]), 'min_z':min(pts[:,2])}

def get_center(pts):
    bounds = get_minmax(pts)
    center = np.array([(bounds['max_x']+bounds['min_x'])/2, 
                       (bounds['max_y']+bounds['min_y'])/2, 
                       (bounds['max_z']+bounds['min_z'])/2])  
    return center

def get_lwh(minmax):
    l1 = minmax['max_x'] - minmax['min_x']
    l2 = minmax['max_y'] - minmax['min_y']
    return np.array([max(l1,l2),min(l1,l2), minmax['max_z'] - minmax['min_z']]) # l,w,h

def get_gt_for_zero_yaw(pts, openpcdet_format=False, o3d_format=False):
    bounds = get_minmax(pts)     
    dims = get_lwh(bounds)
    center = np.array([(bounds['max_x']+bounds['min_x'])/2, 
                       (bounds['max_y']+bounds['min_y'])/2, 
                       (bounds['max_z']+bounds['min_z'])/2])   

    # return 3d box points for o3d box construction
    bbox3d = np.array([
        [bounds['max_x'], bounds['max_y'], bounds['max_z']],
        [bounds['max_x'], bounds['min_y'], bounds['max_z']],
        [bounds['min_x'], bounds['max_y'], bounds['max_z']],
        [bounds['min_x'], bounds['min_y'], bounds['max_z']],
        [bounds['max_x'], bounds['max_y'], bounds['min_z']],
        [bounds['max_x'], bounds['min_y'], bounds['min_z']],
        [bounds['min_x'], bounds['max_y'], bounds['min_z']],
        [bounds['min_x'], bounds['min_y'], bounds['min_z']]
    ])
    if o3d_format:
        o3dvec_bbox = o3d.utility.Vector3dVector(bbox3d)
        o3dbox = o3d.geometry.OrientedBoundingBox().create_from_points(o3dvec_bbox)
        o3dbox.color = np.array([1,0,0])
        return o3dbox
    else:
        return {'bbox':bbox3d, 'center':center, 'dims':dims} 
    
def get_scaling_factor(pts, min_car_width=1.55, max_car_width=2.15):
    bounds = get_minmax(pts)
    dims = get_lwh(bounds)
    car_width = np.random.uniform(min_car_width,max_car_width)
    scaling_factor = car_width/dims[1]
    return scaling_factor

def load_shapenet(obj_file, num_pts=300000):
    v,f,n = pcu.load_mesh_vfn(obj_file)
    
    # Shapenet forward is -z, here we change to +z to fit right hand rule convention (convention for most lidars)
    # Detection datasets define car length as x, width as y, and height as z. Forward is +x
    v[:,2] *= -1
    
    # Here we re-orient shapenet coordinate system of [z,x,y] to fit typical lidar convention of [x,y,z]
    v = v[:,[2,0,1]]     
    
    # dense random sampling
    fi, bc = pcu.sample_mesh_random(v, f, num_pts)
    pts = pcu.interpolate_barycentric_coords(f, fi, bc, v)
    
    # ShapeNet is centered but it's slightly off 0,0,0
    bounds = get_minmax(pts)
    center = np.array([(bounds['max_x']+bounds['min_x'])/2, 
                       (bounds['max_y']+bounds['min_y'])/2, 
                       (bounds['max_z']+bounds['min_z'])/2]) 
    return pts - center

def cylindrical2cart(points):
    r, azimuth, z = points[:,0], points[:,1], points[:,2]
    x = r*np.cos(azimuth)
    y = r*np.sin(azimuth)
    return np.concatenate((x[:,np.newaxis],y[:,np.newaxis],z[:,np.newaxis]), axis=1)

def sph2cart(points):
    r, azimuth, elevation = points[:,0], points[:,1], points[:,2]
    x = r*np.sin(elevation)*np.cos(azimuth)
    y = r*np.sin(elevation)*np.sin(azimuth)
    z = r*np.cos(elevation)
    return np.concatenate((x[:,np.newaxis],y[:,np.newaxis],z[:,np.newaxis]), axis=1)

def cart2sph(points):
    # elevation is up-down, azimuth is left-right
    x,y,z = points[:,0], points[:,1], points[:,2]
    r = np.linalg.norm(points, axis=1)
    elevation = np.arctan2(np.linalg.norm(points[:,:2],axis=1),z) # also = np.arccos(z/r)
    azimuth = np.arctan2(y,x) # theta in radians, lidar is 360 degrees hence the 3.14 to -3.14 values
    
    return np.concatenate((r[:,np.newaxis],azimuth[:,np.newaxis], elevation[:,np.newaxis]), axis=1)

def generate_ncoords_by_azimuth_bins(nbins=16, min_dist=5, max_dist=70, z_mu=0.8, z_sigma=0.3):
    azbin_size = 2*np.pi/nbins
    azbin_edges = [-np.pi+i*azbin_size for i in range(nbins+1)]

    cyl = []
    for i in range(nbins):
        rand_dist = np.random.uniform(min_dist,max_dist)
        az_bin = [azbin_edges[i], azbin_edges[i+1]]
        sampled_az = np.random.normal(np.mean(az_bin), azbin_size/4)
        cyl.append([rand_dist, sampled_az])

    z = np.random.normal(loc=z_mu, scale=z_sigma, size=[len(cyl)])
    return cylindrical2cart(np.hstack([np.array(cyl), z[...,np.newaxis]]))


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
data_dir = '/SEE-MTDA/data/shapenet/pcn/VC_format/train/complete/*.pcd'
pcd_files = sorted(glob.glob(data_dir))

sel_idx = 0
complete_path = pcd_files[sel_idx]
partial_path = complete_path.replace('complete', 'partial')
complete_pcd = o3d.io.read_point_cloud(complete_path)
# complete_pcd.paint_uniform_color([0.9,0.9,0.9])
partial_pcd = o3d.io.read_point_cloud(partial_path)


In [5]:
o3d.visualization.draw_geometries([partial_pcd])

In [68]:
numpts = []
for idx in range(len(pcd_files)):
    complete_path = pcd_files[idx]
    partial_path = complete_path.replace('complete', 'partial')
    partial_pcd = o3d.io.read_point_cloud(partial_path)
    numpts.append(len(partial_pcd.points))

In [69]:
max(numpts)

2048

In [52]:
from tqdm import tqdm
from pathlib import Path
import pickle

dataset_size = 1000
subset = pcd_files[:dataset_size]

save_dir = Path('/SEE-MTDA/data/shapenet/VC/pcn-mini')
save_dir.mkdir(exist_ok=True, parents=True)

for idx in tqdm(range(len(subset)), total=len(subset)):
        
    # Load points of a single model
    complete_path = pcd_files[idx]
    partial_path = complete_path.replace('complete', 'partial')
    gt_pts = np.asarray(o3d.io.read_point_cloud(complete_path).points)
    partial_pts = o3d.io.read_point_cloud(partial_path)
    
    gtbox = get_gt_for_zero_yaw(gt_pts)
    data = {'bbox': gtbox['bbox'],
            'partial': partial_pts,
            'complete': convert_to_o3dpcd(gt_pts),
            'heading': 0,
            'centroid': gtbox['center'],
            'dims': gtbox['dims']}
    
    model_id = complete_path.split('/')[-1].split('_')[1]
    view_id = int(complete_path.split('/')[-1].split('.')[0].split('_')[-1])
    
    # Save partial, complete, labels
    partial_dir = save_dir / 'partial' / model_id
    partial_dir.mkdir(exist_ok=True, parents=True)
    
    complete_dir = save_dir / 'complete' / model_id
    complete_dir.mkdir(exist_ok=True, parents=True)
    
    label_dir = save_dir / 'label' / model_id
    label_dir.mkdir(exist_ok=True, parents=True)
    
    o3d.io.write_point_cloud(str(partial_dir / f'{view_id:03d}.pcd'), data['partial'])
    o3d.io.write_point_cloud(str(complete_dir / f'{view_id:03d}.pcd'), data['complete'])
    with open(str(label_dir / f'{view_id:03d}.pkl'), 'wb') as f:
        label = {'bbox': data['bbox'], 'dims': data['dims'], 'heading': data['heading'], 'centre': data['centroid']}
        pickle.dump(label, f)
            
# Chown             
import os

os.system(f"cd {str(save_dir)} && ls complete > model_ids.txt")
os.system(f"cd {str(save_dir.parent)} && chown -R 1005:1005 {str(save_dir.stem)}")

100%|███████████████████████████████████████████| 1000/1000 [00:16<00:00, 59.98it/s]


0